# F# Advent Calendar 2022

## Using F# to help solve Wordle

This is my contrinution to the F# Advent Calendar 2022, which is a relatively short post on using F# to help solve Wordle.
For those who've been living under a rock this year, Wordle is a word guessing game created by Josh Wardle, and now owned by the NY Times.

The game involves iteratively attempting to guess a secret word, with each guess 'scored' against this secret word. If the guess word's letter is in the correct position in the secret word, then that letter is scored as Green. If it's in the word but not in the correct position then Yellow, and Grey if it's not contained in the word.  

You continue to guess until the game is won - where each letter is scored Green - or you run out out attempts.  You only get six attempts to guess the word.

 I found a dictionary of words on the internet, so will use this in order to create a collection of five letter words.  For those of you familiar with wordle, you'll know that the way Josh structured the game, is that that the Worlde is selected from a set of 2315 words, but each word you guess is checked to ensure that it exists in a much larger population of words (approx 11,000).  This may have changed somewhat since the NY Times bought wordle from Josh, and also since they now use a dedicated word setter, rather than using Josh's original list. You can scrape these lists from the JavaScript source code, however in order to not spoil things, I will use my internet downloaded set of words. The stats don't change much between the wordle set of words and the internet set of words - as you can see below.

In [ ]:
#r "nuget: XPlot.Plotly.Interactive"
open XPlot.Plotly

### Part 1 - Analysing five letter words
First of all, some useful functions in order to load the letters from the file.

In [ ]:
open System.IO

let rec listToString l =
    match l with
    | [] -> ""
    | head :: tail -> head.ToString() + listToString tail

let sortString s =
    s |> Seq.sort |> Seq.toList |> listToString

let fiveLetterWords fileName = 
    fileName 
    |> File.ReadAllLines 
    |> Seq.filter (fun w -> w.Length = 5)

let words = fiveLetterWords @"../words.txt"

After loading five letter words from the dictionary file, the first thing we want to do is to determine the best starting word to use.  Much has been written about the best opening word or set of words, and my method is quite simple compared to the more advanced ones (see https://www.3blue1brown.com/lessons/wordle ).

However, a good starting point is to use some frequency analysis, i.e. to derive the best starting word from the most frequent sets of letters that appear in five letter words.

Below I process the list of five letters words, to create a sequence of letters and the count of occurances of that letter within our set of words.  Using F# is wonderfully straightforward, with the collect and countBy functions.

In [ ]:
type Frequency = {Letter: char; Frequency: int}

let frequencies = 
    words 
    |> Seq.collect id 
    |> Seq.countBy id 
    |> Seq.sortByDescending snd

let frequenciesMap =
    frequencies
    |> Map.ofSeq

let vowels = ['a'; 'e'; 'i'; 'o'; 'u']

frequencies |> Seq.take 10 |> Seq.map (fun (l, f) -> {Letter =  l; Frequency = f})

In [ ]:
let x = frequencies |> Seq.toList |> List.map fst
let y = frequencies |> Seq.toList |> List.map snd
let blue = "#447adb"
let red = "#db5a44"
let markers = x |> List.map (fun l -> if List.contains l vowels then red else blue)

// this bar chart doens't seem to render in saved notebooks.
Bar(x = x, y = y, marker = Marker(color = markers)) 
|> Chart.Plot 
|> Chart.WithTitle "Letter Frequencies"
|> Chart.WithXTitle "Letters with vowels highlighted"
printfn "For somereason the char doesn't show in saved notebooks."

Surpringly the letter S, is the most frequent, just pipping E to the top spot, although I expected given E to be high given it is the most frequent letter in the English language.

Now that we have some letter frequencies, lets see if we can find a word in the dictionary set, that uses the top five letters.

The five most frequent letters in frequency order are: 'A', 'E', 'S', 'R', 'O'.  

Let's try to find a word that uses these letters.

One way of doing this, is to create an aggregate score for each word, based on the frequency score of each individual letter.

You really don't want to choose an inintial guess word that contains the same letter, as this doesn't reduce your search space, so I've penalised words that repeat letters.

In [ ]:
type WordScore = {Word: string; Score: int}

let letterScores =
    words
    |> Seq.map (fun word ->
        let penalty = 
            word 
            |> Seq.groupBy id 
            |> Seq.fold (fun state (f, s) ->  state * Seq.length s) 1
        let freq = 
            word 
            |> Seq.fold (fun state letter -> state + Map.find letter frequenciesMap) 0
        word, freq / penalty)
    |> Seq.sortByDescending snd

letterScores |> Seq.take 10 |> Seq.map (fun (l, f) -> {Word = l; Score = f})

Above, are listed the top 10 words, i.e. words that use the most frequent letters, where the letters are all distinct.

So in this case, I've chosen my first word that I will use in Wordle, as "AROSE".  Many people chose "AROSE" as a starting word, probably undertaking the same (basic) analysis.

It's been proven since, that "AROSE" isn't the best starting word, but as I've used it since February, I've decided to stick with it for now.

Now there is a choice, if you play in 'hard mode', which means that you have to reuse letters scored in the previous round, then there is little point finding a generic second word.

However, if you are not using 'hard mode', then it may well be worthwhile finding a second word.  

My thinking here, is that I should choose the next word that scores highly, but doesn't use the same letters are AROSE.

In [ ]:
let aroseSet = "arose" |> Set.ofSeq

let secondWord = 
    letterScores
    |> Seq.find (fun (word, _) ->
        word 
        |> Set.ofSeq    
        |> Set.intersect aroseSet 
        |> Set.count 
        |> (fun count -> count = 0))    
        
secondWord |> (fun (l, f) -> {Word = l; Score = f})

Therefore my next word will be "UNLIT".

Using "AROSE" and "UNLIT" while in 'easy mode' did well for me.  After about a 100 or so games, I have a mode of 3 (see stats page), although I didn't have any luck guessing below 3, obviously.

But the What's app group that I play Worlde with, decided to move to use 'hard mode', so I could not longer use "UNLIT" as my second word, as I have to choose a second word which contains letters from "AROSE" in the right position, as given by the scoring algorithm.

### Positional Frequencies
Let's now look at individual letter frequencies in each position, i.e. what's the most common first, last letter etc.

We can process the word list again, but this time splitting each word into individual letters, and then creating a count of letters in the first, second etc positions.  This will help, after entering the first and second words, in order to place Yellow letters into the correct positions.

In [ ]:
let letterPositionFrequency (words: string seq) = 
    words
    |> Seq.map (fun w -> w.ToCharArray() |> Array.indexed)
    |> Seq.collect id
    |> Seq.groupBy fst
    |> Seq.map (fun (p, cs) -> p, cs |> Seq.countBy snd |> Seq.sortByDescending snd)

let mostFrequentLetterPosition words = 
    words 
    |> letterPositionFrequency
    |> Seq.map (fun (p, lfs) -> p, lfs |> Seq.head |> fst)

words |> mostFrequentLetterPosition

You can see that the most frequent letters in each position are: S, A, A, E, S, which isn't that surprising, given the first frequency analysis we did.

Although this is using a far larger set of 5 letter words that the one used in Wordle.

So let's try this with the actual wordle dataset (this was taken last April, so may well have changed now)

In [ ]:
let wordleWords = File.ReadAllLines @"../Python/wordle_words.txt"
wordleWords |> Seq.length

In [ ]:
wordleWords |> mostFrequentLetterPosition

In [ ]:
wordleWords |> letterPositionFrequency

In [ ]:
let filterFrequenciesOn letter = 
    wordleWords
    |> letterPositionFrequency
    |> Seq.map (fun (p, cs) -> cs |> Seq.filter (fun (l, _) -> l = letter) |> Seq.head)

filterFrequenciesOn 'y'

In [ ]:
let letterPositionFrequencyFilter position letter = 
    wordleWords
    |> Seq.map (fun w -> w.ToCharArray() |> Array.indexed)
    |> Seq.filter (fun cs -> cs.[position - 1] |> snd = letter)
    |> Seq.collect id
    |> Seq.groupBy fst
    |> Seq.map (fun (p, cs) -> p, cs |> Seq.countBy snd |> Seq.sortByDescending snd)

letterPositionFrequencyFilter 5 'y'

As you can see above, position frequency analysis can yield useful results. The letter Y occurs far more frequently at the end of a five letter word, then any other position.

Filtering on words ending in Y, you can see that the penultimate letter in the word, has a high chance of being L, T, D, R, K, which indicates that the last two letters of a word ending in Y could be, LY, TY, DY, RY.

### Scoring

Let's now create a scorer function, we can use in any strategy that we might employ (and my own wordle game, see below).

This is a function that takes a guess word and the actual wordle, and returns for each guessed letter, whether it is Yellow (in the wordle but not in correct position), Green (in the correct position), Grey (not in the wordle.)

In order to score correctly (see point below), you need to keep track of the number of Yellow words in your guess word, and reduce this number each time you score a letter as yellow. For this I've created my own Counter type, which acts in a similar way to the Counter function in Python (which I originally used for my analysis).

In [ ]:
type AnswerMask = Green | Yellow | Grey

module Counter =
    let createCounter items =
        items
        |> List.filter (fun (a, g) -> a <> g)
        |> List.map fst
        |> List.countBy id
        |> Map.ofList

    let countOf counter item =
        match Map.tryFind item counter with
        | Some c -> c
        | None -> 0

    let updateCount counter item =
        match Map.tryFind item counter with
        | Some c -> Map.add item (c - 1) counter
        | None -> counter

let scoreGuess actual guess =

    let letters = Seq.zip actual guess |> Seq.toList

    let folder ((count, mask): Map<'a,int> * AnswerMask list) (a, g) =
        if a = g then
            count, Green :: mask
        elif Seq.contains g actual && Counter.countOf count g > 0 then
            Counter.updateCount count g, Yellow :: mask
        else
            count, Grey :: mask

    List.fold folder (Counter.createCounter letters, []) letters |> snd |> List.rev

Now scoring, the guess 'arose' against the wordle 'favour', we see that the answer mask returned is Yellow, Yellow, Yellow, Grey, Grey.

In [ ]:
type ScoreResult = {Wordle: String; Guess: string; Result:AnswerMask list}

let getScoreResult (wordle:string) (guess: string) = 
     wordle.ToUpper(), guess.ToUpper(), scoreGuess wordle guess

let tests = 
    [
        "favor", "arose"
        "favor", "ratio"
        "favor", "carol"
        "favor", "vapor"
        "arose", "speed"
        "treat", "speed"
    ]

tests 
|> List.map (fun (wordle, guess) -> getScoreResult wordle guess)
|> List.map (fun (w, g, r) -> {Wordle = w ; Guess = g; Result = r})

As I mentioned above, one key point is the handling of a double Yellow letter, which many people in examples that I've seen, get wrong.

If the wordle is "AROSE" and your guess is "SPEED", then you should score it as Yellow, Grey, Yellow, Grey, Grey, with only the first instance of "E" in the "SPEED" being scored as a Yellow, the second instance is just a Grey.

As an example, many do the following:

In [ ]:
let actual = "arose"
let guess = "speed"
let dodgyScorer (actual:string) (guess: string) = 
    let letters = Seq.zip actual guess |> Seq.toList
    let rec masker ls mask =
        match ls with
        | [] -> mask
        | (a, g) :: t when a = g -> masker t (Green :: mask)
        | (a, g) :: t when actual.Contains(g |> string) -> masker t (Yellow :: mask)
        | h :: t -> masker t (Grey :: mask)
    actual, guess, masker letters [] |> List.rev

dodgyScorer actual guess |> fun (w, g, r) -> {Wordle = actual ; Guess = guess; Result = r}

In [ ]:
tests 
|> List.map (fun (wordle, guess) -> dodgyScorer wordle guess)
|> List.map (fun (w, g, r) -> {Wordle = w ; Guess = g; Result = r})

As you can see, in the final two words, the dodgyScorer function gets the score wrong - so it's good to add these sets of words to any unit tests.

At this point, I had planned on running the simulation that I wrote in Python - it plays Wordle for each 2315 words employing custom strategies - using Fable to Python.  Alas, as with most things around Christmas, I didn't leave enough time to get this done.

Maybe if you read this in the New Year, I would have done this.

### Part 2 Putting it all together.

This should have been a separate blog post, however I might as well include it here. Earlier this year, I wrote my own version of Wordle, to help my daughter with her Phonics at school.  It's basically the same Wordle, game as described above, but I also provide a Phonic hint for each word, i.e. if the Wordle is "GREAT", then the Phonic hint would be /AE/ corresponding to the sound made by the Grapheme 'EA' in GREAT.

I wrote this in Fable.Lit, copying a Math Game that Aaron made for his daugher (https://aaronmu.github.io/MathGame/).

I've done very little in HTML prior to this exercise, and nothing in CSS, and knowing very little, I was surprised how quickly I could achieve something in F# and Fable.

Here's my version of Wordle, https://ioancw.github.io/AureliaWordle/.  It's been optimised for the non pro/max versions of the iPhones, so hopefully it will render OK on whatever device you are using.  

The code is here is you want to have a look, https://github.com/ioancw/AureliaWordle.

All for now, hopefully I will have time to expand on the above in the future.

Thanks to Sergey Tihon for setting up the F# Advent Calendar, and his F# Weekly blog posts.